## 中文评估


In [5]:
%pip install -Uq dspy-ai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy 0.1.5 requires dspy-ai==2.4.5, but you have dspy-ai 2.4.13 which is incompatible.


In [1]:
file_path = "../.data/all_raretrible.txt"
data = open(file_path, "r", encoding="utf-8").read().split("\n")
data = [x.strip().split(",") for x in data if len(x) > 0]
data = [[x[0], ",".join(x[1:])] for x in data]
print(data[0:2])

[['感激', '好的,那我谢谢您了。'], ['中性', '没有词汇课的优惠券吗?']]


In [2]:
import pandas as pd

columns=["label", "text",]
# 转换 data 到 pandas DataFrame
df = pd.DataFrame(data, columns=columns)


In [3]:
from dspy.datasets import DataLoader

# 创建 dspy.datasets 对象
dl = DataLoader()
# fields 选择指定列 input_keys
all_dataset = dl.from_pandas(
    df,
    fields=columns,
    input_keys=("text",),
)
print(all_dataset[0:2])

[Example({'label': '感激', 'text': '好的,那我谢谢您了。'}) (input_keys={'text'}), Example({'label': '中性', 'text': '没有词汇课的优惠券吗?'}) (input_keys={'text'})]


In [4]:
from collections import defaultdict
import random


def stratified_sample(from_dataset, label_attr, *dataset_sizes):
    label_groups = defaultdict(list)
    for item in from_dataset:
        label = getattr(item, label_attr)
        label_groups[label].append(item)

    result_sets = [[] for _ in dataset_sizes]

    for label, items in label_groups.items():
        random.shuffle(items)
        label_sizes = [size // len(label_groups) for size in dataset_sizes]
        start = 0
        for i, size in enumerate(label_sizes):
            end = start + size
            result_sets[i].extend(items[start:end])
            start = end

    all_items = [item for items in label_groups.values() for item in items]
    for i, (result_set, target_size) in enumerate(zip(result_sets, dataset_sizes)):
        shortage = target_size - len(result_set)
        if shortage > 0:
            result_sets[i].extend(random.sample(all_items, shortage))

    return result_sets


In [5]:
train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
from collections import Counter

train_set, test_set, sample_set = stratified_sample(all_dataset, "label", 20, 50, 100)
for dataset in [train_set, test_set, sample_set]:
    label_counts = Counter(item.get("label") for item in dataset)
    print(label_counts)

Counter({'高兴': 4, '感激': 3, '惊讶': 3, '抱怨': 3, '生气': 3, '中性': 2, '焦急': 2})
Counter({'中性': 8, '感激': 7, '焦急': 7, '惊讶': 7, '高兴': 7, '抱怨': 7, '生气': 7})
Counter({'感激': 15, '生气': 15, '中性': 14, '焦急': 14, '惊讶': 14, '高兴': 14, '抱怨': 14})


In [7]:
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.show_error()

You are a customer service quality inspection expert., sentiment analysis from Question

You should pay special attention to the emotions of customers being angry and complaining.

---

Follow the following format.

Question: list of input text
Answer:
    Return format: a list of "Index, Emotion Category, Emotion Subcategory, Emotion Subcategory Score"
    Emotion Subcategory Score (0-1.00).
    Example: 
        1, pessimistic, complaining, 0.90
        2, optimistic, happy, 0.75
Please choose emotion categories and subcategories only from the following options:
    Emotion categories: ["pessimistic", "optimistic", "neutral"]
    Emotion subcategories: ['neutral', 'surprised', 'thankful', 'complaining', 'urgent','anxious', 'angry', 'happy']

Here is some examples

---

Question:
1: 好的谢谢[亲]
Answer:
1,optimistic,thankful,0.90

---

Question: 
1: 在吗?回复太慢了
Answer:
1,pessimistic,complaining,0.85

---

Question: 
1: 都要举报
Answer:
1,pessimistic,angry,0.95

---

Question: 
1: 我一直都是用的这个号,怎么突然就

# 测试印尼语


In [10]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)

In [11]:
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set)
ret = trans.do()
print(ret.content)

把下面内容翻译成 印尼语: 
1: 哦!我懂了好的谢谢
2: 我看看啊,谢谢
3: 好的谢谢你,新年快乐哈
4: 谢谢亲啦[emoji050]
5: 明白啦,谢谢啦亲爱哒
6: 支付宝收到。谢谢
7: 感谢你给我帮助,不打扰你了
8: 突然
9: 江门市外海镇
10: 不要你说那么多,我就只想晓得我这一单货,益生菌的这个
11: 另外,您说博影点数和人民币不是相同换算单位,请告知换算汇率,谢谢
12: 您好,查询一下
13: 有发短信么
14: 账户冻结了,怎么办
15: 帮忙催下谢谢
16: 亲,快些哈
17: 审核也太慢了
18: 希望你们帮我解决一下这个问题,谢谢哈!
19: 代理看不到订单心里很着急
20: 麻烦稍快些
21: Baik, terima kasih. Jadi, mohon acu-acu untuk mempersiapkan dengan cepat
22: 然后一直扣费?
23: 我只用了两个小时都没到怎么扣了六块
24: 明明抢到了呀
25: 哇,你们厉害了
26: 两个东西都没有发货?
27: 里面的人怎么都是做鸡的
28: 还要收费???
29: 好,好,很好
30: 我觉得你挺帅
31: 哈哈哈你好巧巧
32: 太期待了
33: 好萌,哈哈
34: :我爱你
35: 我爱你么么哒
36: 你们这边遇到这样问题这样解决的,又没有联系方式,只能留言,他们设的系统回复解决不了问题
37: 电脑能连接上,手机连接不上怎么回事
38: 别人讲的不爱听
39: 每次都是这样,用着还得手动大屏,很烦啊!重度强迫症受不了啊!
40: 太难受了,天天在家又不能出门,玩游戏还出现问题
41: 网速慢到打不开
42: 很多板块都是连锁反应很慢
43: 这样又丢件,又不索赔给我们
44: 规定你妈
45: 不,我他妈第三次我选择了提现到银行卡
46: 滚鸡巴蛋
47: 我只能投诉你们了
48: 你们是放高利贷的
49: 你们就是诈骗平台
50: 哦哦,两天了,一直审核中的呢

1: Oh! Saya mengerti, baiklah terima kasih.
2: Aku cek dulu, terima kasih.
3: Baiklah, terima kasih, selamat tahun baru.
4: Terima kasih, ka

In [16]:
trans.update_set()
for item in test_set:
    print(item)

Example({'label': '感激', 'text': 'Oh! Saya mengerti, baiklah terima kasih.', 'source': '哦!我懂了好的谢谢'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Aku cek dulu, terima kasih.', 'source': '我看看啊,谢谢'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Baiklah, terima kasih, selamat tahun baru.', 'source': '好的谢谢你,新年快乐哈'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Terima kasih, kak [emoji050].', 'source': '谢谢亲啦[emoji050]'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Saya mengerti, terima kasih, cinta sayang.', 'source': '明白啦,谢谢啦亲爱哒'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Alipay sudah menerima. Terima kasih.', 'source': '支付宝收到。谢谢'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Terima kasih atas bantuanmu, jangan ganggu.', 'source': '感谢你给我帮助,不打扰你了'}) (input_keys={'text'})
Example({'label': '中性', 'text': 'Tiba-tiba.', 'source': '突然'}) (input_keys={'text'})
Example({'label': '中性', 'text': 'Daerah Waihai, Kota Jiangmen.', 'source'

In [ ]:
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.show_main_error()

In [18]:
ana.show_error()

0.6
Example({'label': '感激', 'text': 'Baiklah, terima kasih, selamat tahun baru.', 'source': '好的谢谢你,新年快乐哈', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Alipay sudah menerima. Terima kasih.', 'source': '支付宝收到。谢谢', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'Terima kasih atas bantuanmu, jangan ganggu.', 'source': '感谢你给我帮助,不打扰你了', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '中性', 'text': 'Ada SMS?', 'source': '有发短信么', 'pred_label': '焦急', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '焦急', 'text': 'Bantu pergi ke sana, terima kasih.', 'source': '帮忙催下谢谢', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '焦急', 'text': 'Kak, tinggal cepatlah.', 'source': '亲,快些哈', 'pred_label': '中性', 'main': '中性', 'score': '0.60'}) (input_keys={'text'})
Example({'label': '

In [22]:
ana.show_main_error()
for item in ana.data_set:
    print(item['label'],item['pred_label'],item['source'],)

0.0
感激 感激 哦!我懂了好的谢谢
感激 感激 我看看啊,谢谢
感激 中性 好的谢谢你,新年快乐哈
感激 感激 谢谢亲啦[emoji050]
感激 感激 明白啦,谢谢啦亲爱哒
感激 中性 支付宝收到。谢谢
感激 中性 感谢你给我帮助,不打扰你了
中性 中性 突然
中性 中性 江门市外海镇
中性 中性 不要你说那么多,我就只想晓得我这一单货,益生菌的这个
中性 中性 另外,您说博影点数和人民币不是相同换算单位,请告知换算汇率,谢谢
中性 中性 您好,查询一下
中性 焦急 有发短信么
中性 中性 账户冻结了,怎么办
焦急 中性 帮忙催下谢谢
焦急 中性 亲,快些哈
焦急 焦急 审核也太慢了
焦急 感激 希望你们帮我解决一下这个问题,谢谢哈!
焦急 焦急 代理看不到订单心里很着急
焦急 焦急 麻烦稍快些
焦急 中性 好谢谢。那就帮我尽快催促一下吧
惊讶 中性 然后一直扣费?
惊讶 抱怨 我只用了两个小时都没到怎么扣了六块
惊讶 中性 明明抢到了呀
惊讶 中性 哇,你们厉害了
惊讶 中性 两个东西都没有发货?
惊讶 中性 里面的人怎么都是做鸡的
惊讶 抱怨 还要收费???
高兴 中性 好,好,很好
高兴 感激 我觉得你挺帅
高兴 中性 哈哈哈你好巧巧
高兴 中性 太期待了
高兴 中性 好萌,哈哈
高兴 感激 :我爱你
高兴 感激 我爱你么么哒
抱怨 抱怨 你们这边遇到这样问题这样解决的,又没有联系方式,只能留言,他们设的系统回复解决不了问题
抱怨 抱怨 电脑能连接上,手机连接不上怎么回事
抱怨 抱怨 别人讲的不爱听
抱怨 抱怨 每次都是这样,用着还得手动大屏,很烦啊!重度强迫症受不了啊!
抱怨 抱怨 太难受了,天天在家又不能出门,玩游戏还出现问题
抱怨 抱怨 网速慢到打不开
抱怨 抱怨 很多板块都是连锁反应很慢
生气 抱怨 这样又丢件,又不索赔给我们
生气 中性 规定你妈
生气 中性 不,我他妈第三次我选择了提现到银行卡
生气 抱怨 滚鸡巴蛋
生气 抱怨 我只能投诉你们了
生气 抱怨 你们是放高利贷的
生气 抱怨 你们就是诈骗平台
焦急 抱怨 哦哦,两天了,一直审核中的呢


## 泰语

印尼、泰、马来、菲律宾、粤语、日语、韩语


In [ ]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="泰语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)

In [38]:
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

You are a customer service quality inspection expert., sentiment analysis from Question

You should pay special attention to the emotions of customers being angry and complaining.

---

Follow the following format.

Question: list of input text
Answer:
    Return format: a list of "Index, Emotion Category, Emotion Subcategory, Emotion Subcategory Score"
    Emotion Subcategory Score (0-1.00).
    Example: 
        1, pessimistic, complaining, 0.90
        2, optimistic, happy, 0.75
Please choose emotion categories and subcategories only from the following options:
    Emotion categories: ["pessimistic", "optimistic", "neutral"]
    Emotion subcategories: ['neutral', 'surprised', 'thankful', 'complaining', 'urgent','anxious', 'angry', 'happy']

Here is some examples

---

Question:
1: 好的谢谢[亲]
Answer:
1,optimistic,thankful,0.90

---

Question: 
1: 能帮我加急处理吗,项目上的工人情绪都很大,谢谢
Reasoning: Urgent handling indicates that the customer wants to resolve the issue as soon as possible, and strong emot

In [39]:
for item in ana.data_set:
    print(item['label'],item['pred_label'],item['source'],)

感激 感激 感谢,祝新年快乐??
感激 中性 好的~谢谢~
感激 中性 已经到了,感谢
感激 中性 谢谢,刚才找了一遍没找到
感激 中性 嗯嗯,谢谢您
感激 中性 好的谢谢,亲!
感激 中性 谢谢你,真细心
中性 中性 你这三期套是什么意思
中性 抱怨 为什么有些词语发不出去?
中性 抱怨 之前有打过电话来,但都显示推销什么的,所以一直没接
中性 抱怨 我这卡停机了吗?
中性 抱怨 你浪费了这半天的都痛时间
中性 中性 就是病理是吗
中性 中性 我故意不投了
焦急 焦急 能帮我加急处理吗,项目上的工人情绪都很大,谢谢
焦急 中性 没试过那个平台放款这么久的
焦急 焦急 亲爱的,能快点吗
焦急 抱怨 这个退订很久了,怎么还没有退款啊
焦急 焦急 尽量催一下吧,谢谢
焦急 焦急 因为我想早点把积分退还过来,我就快点把东西买完,然后就快点收到。请问可以尽量在这周内处理完么?谢谢
焦急 焦急 尽量催一下哦
惊讶 中性 今天一看吓死还在计费
惊讶 中性 就是坑坑洼洼的,你可能看到??
惊讶 抱怨 半个小时还没变?
惊讶 抱怨 这是真的不回我了?
惊讶 中性 [emoji039]这么厉害的啊
惊讶 中性 里面的人怎么都是做鸡的
惊讶 中性 里面的人怎么都是做鸡的
高兴 抱怨 要向你学习全心全意为人民服务的精神。又快又好
高兴 中性 哈哈哈哈哈新年快乐哦
高兴 感激 我爱你燕子
高兴 感激 我非常满意
高兴 中性 哈哈哈哈可以的亲
高兴 中性 好的好的哈哈
高兴 中性 出来了哈哈哈哈
抱怨 抱怨 订单也打不开,这都过去半个小时了
抱怨 抱怨 不然过年太糟心了
抱怨 抱怨 跑了几趟,感觉很麻烦,你直接帮我扣钱行了
抱怨 抱怨 一直没信息回复
抱怨 抱怨 真的没有地方可以还啊
抱怨 抱怨 不发货又不让退货
抱怨 抱怨 这玩意根本看不明白
生气 中性 你妈那个比的
生气 中性 不给我暂停,新年死全家
生气 抱怨 凭什么你们说怎么还就怎么还
生气 抱怨 我看了评论都说平台不靠谱,但是还是充值试了下,感觉这个保证金真的非常坑
生气 抱怨 快过年还要老子骂你们,垃圾
生气 抱怨 你要我看什么狗屁公告
生气 中性 你们的电脑后台是不是我打爱心,你们只会出现问号?
中性 抱怨 你们这机器还什么未连网络


In [27]:
ana.show_error()

0.66
Example({'label': '感激', 'text': 'ยังติดหรอย ขอบคุณครับ', 'source': '好的~谢谢~', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'ถึงแล้ว ขอบคุณครับ', 'source': '已经到了,感谢', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'ขอบคุณ ฉันหาแล้วไม่เจอ', 'source': '谢谢,刚才找了一遍没找到', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'อ่อยๆ ขอบคุณครับ', 'source': '嗯嗯,谢谢您', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'ยังติดหรอย ขอบคุณครับ หนูนายแล้ว', 'source': '好的谢谢,亲!', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '感激', 'text': 'ขอบคุณครับ คุณมีใจในใน', 'source': '谢谢你,真细心', 'pred_label': '中性', 'main': '中性', 'score': '0.70'}) (input_keys={'text'})
Example({'label': '中性', 'text': 'ทำไมบางคำไม่สามารถส่งออกได้เลยครับ', 'sour

In [30]:
for item in ana.data_set:
    anti = ana.is_anti_main_cat(item['main'], item['label'])
    print(item['label'],item['pred_label'],ana.sub_cover.to_cn(item['label'].strip()), anti,item['source'],)

正面 感激 正面 False 感谢,祝新年快乐??
中性 感激 正面 False 好的~谢谢~
中性 感激 正面 False 已经到了,感谢
中性 感激 正面 False 谢谢,刚才找了一遍没找到
中性 感激 正面 False 嗯嗯,谢谢您
中性 感激 正面 False 好的谢谢,亲!
中性 感激 正面 False 谢谢你,真细心
中性 中性 中性 False 你这三期套是什么意思
负面 中性 中性 False 为什么有些词语发不出去?
中性 中性 中性 False 之前有打过电话来,但都显示推销什么的,所以一直没接
中性 中性 中性 False 我这卡停机了吗?
中性 中性 中性 False 你浪费了这半天的都痛时间
中性 中性 中性 False 就是病理是吗
中性 中性 中性 False 我故意不投了
正面 焦急 负面 False 能帮我加急处理吗,项目上的工人情绪都很大,谢谢
中性 焦急 负面 False 没试过那个平台放款这么久的
中性 焦急 负面 False 亲爱的,能快点吗
负面 焦急 负面 False 这个退订很久了,怎么还没有退款啊
中性 焦急 负面 False 尽量催一下吧,谢谢
正面 焦急 负面 False 因为我想早点把积分退还过来,我就快点把东西买完,然后就快点收到。请问可以尽量在这周内处理完么?谢谢
中性 焦急 负面 False 尽量催一下哦
负面 惊讶 负面 False 今天一看吓死还在计费
中性 惊讶 负面 False 就是坑坑洼洼的,你可能看到??
中性 惊讶 负面 False 半个小时还没变?
负面 惊讶 负面 False 这是真的不回我了?
中性 惊讶 负面 False [emoji039]这么厉害的啊
中性 惊讶 负面 False 里面的人怎么都是做鸡的
中性 惊讶 负面 False 里面的人怎么都是做鸡的
负面 高兴 正面 False 要向你学习全心全意为人民服务的精神。又快又好
中性 高兴 正面 False 哈哈哈哈哈新年快乐哦
中性 高兴 正面 False 我爱你燕子
中性 高兴 正面 False 我非常满意
中性 高兴 正面 False 哈哈哈哈可以的亲
中性 高兴 正面 False 好的好的哈哈
中性 高兴 正面 False 出来了哈哈哈哈
中性 抱怨 负面 False 订单也打不开,这都过去半个小时了
负面 抱怨 负

## 马来、


In [40]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="马来语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 马来语: 
1: 谢谢你们了[emoji050]
2: 哦,我没看见有优惠券就问问,谢谢
3: 好的,谢谢美女。
4: 好的,,,了解,谢谢
5: 不需要了!谢谢!你们办事效率高!点赞/:@)
6: 谢了,祝你工作顺利
7: 可以了谢谢你,
8: 我现在有个服务单
9: 交易一直复核是不是审核不通过呀
10: 有惠油吧的详细资料吗
11: 你问下技术是蒜苗原因
12: 餐厅放假了,又闹瘟疫,不知道什么时候开门
13: 你好,我已经购买完成了
14: 那这样产生的费用也需要我个人承担?
15: 别查太久,我也没时间在这等
16: 请尽快,急,谢谢
17: 急,麻烦尽快安排发货,谢谢。
18: 能不能联系到商家,今天能不能发货?这批货真的着急要了
19: 麻烦你尽快帮我安排下去吧,谢谢
20: 怎么还不送货安装
21: 着急,
22: 怎么可能明明不能用?
23: 你还让我给你好评吗?
24: 怎么这样呀
25: 充电宝只充了不到两个小时就没电了?
26: 不是吧?有没有搞错
27: 我的账号怎么不在了
28: 你还让我给你好评吗?
29: ฉันต้องการเรียกให้คุณเรียนรู้จากจิตวิญญาณของการบริการประชาชนที่มีทั้งความเร็วและคุณภาพครับ
30: 再次㊗️您新年快乐
31: 亲爱的新年好的
32: [emoji047]真好
33: 快乐快乐
34: 哈哈哈哈
35: 哦,哈哈
36: 你们售后真麻烦,淘宝提供这些就已经能退了
37: 用得着核实这么久么
38: 白萝卜不仅断了,还压坏了
39: 里面的东西都是乱的[emoji026]
40: 好吧,像街电,只要满了就会有管理人员收走几个,就方便人家还,和借,一看你们怪兽次次都是满的,都不敢借了,怕还起来麻烦,给你们提个小意见,希望能理解我们用户的心里,谢谢
41: 你们的回款也太慢了吧
42: 你们所要得这个授权有点太强人所难了吧
43: 我去尼玛的吧
44: 麻烦你尽快给我处理我问题,性质太恶劣了,必须严惩?
45: 你们在哪里可以投诉呢!
46: 帮我投诉
47: 死了没,没死说话啊
48: 操**
49: 你继续骗我
50: 我在买单侠不欠你们一分钱,早就还清了所有的帐单,怎么回事。

1: Terima kasih

## 菲律宾


In [41]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="菲律宾语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 菲律宾语: 
1: 是的谢谢亲辛苦了
2: 是的,谢谢诶
3: 我明白了,谢谢亲爱的
4: 好的!我试一下,谢谢!
5: 这回没问题了,再次感谢
6: 谢谢你的耐心讲解,拜拜
7: 好的,了解谢谢
8: 麻烦退款一百,满仓导致的扣费。谢谢
9: 还没用,就坏了
10: 跟手电筒似得
11: 不小心删掉了
12: 显示无法打开网页
13: 扣二虎也是一个毛老八
14: 那我沟通下审批人吧谢谢
15: 你们的速度慢
16: 快快快说话
17: 就是尽快发货,谢谢
18: 但这么多天都没有一点进展啊
19: 已经等半月了,太久了
20: 麻烦尽快。我在淋着雨。谢谢姐姐
21: 请帮我反馈,请帮我尽快解决,谢谢
22: 怎么这样呀
23: 我的账号怎么不在了
24: 不交诚意金聊不了嘛不交诚意金聊不了嘛
25: 又可以用了?谢谢啊
26: 吓
27: 今天一看吓死还在计费
28: 什么鬼
29: 好,好,很好
30: 加油加油加油亲爱的
31: 真好,你终于理解了
32: 哈哈哈
33: 哈哈哈哈
34: 要向你学习全心全意为人民服务的精神。又快又好
35: 好,好,很好
36: 我每次问你们都要累死
37: 支付也很慢,长时间网络不能用
38: 问题根本就不可能触发
39: 你们这个在线客服很多截图都发不出去
40: 批量很麻烦
41: 还是不行,唉
42: 心酸啊!
43: 你们这个欺骗行为
44: 产品到期一年多,转让进度像蜗牛,转让额度也没有,首页的新闻是骗子吧
45: 要我骂你?
46: 人都死光光了吗
47: 你们这政策太离谱了
48: 气死我了,早知道就不花钱。全是妈妈级别的。年龄大点就大点吧。还这么丑。我都要吐了。
49: Kalian seperti pemberi pinjaman tinggi suku bunga.
50: 你们开始坑人了,还的方式没有问题,别把问题往客户身上扯,如果还的不对是不会结束订单扣费的,

1: Oo, salamat nang nagdulot ng kahirapan.
2: Oo, salamat na.
3: Naiintindihan ko na, salamat ka, my love.
4: Oo, susubukan ko, salamat.
5: Ngayon wala na proble

## 粤语


In [43]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="粤语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 粤语: 
1: 好哒,好哒,谢谢大大
2: 已经可以了,太谢谢啦
3: 哦哦好滴,谢谢你啦
4: /好的谢谢
5: 好的,谢谢,下次再聊
6: 好的麻烦您
7: 谢谢您!!!
8: 都不敢接
9: 好的,知道了,我的客户还以为是上架了就没事了呢[emoji029]
10: 查查这个认证没
11: 但是我这边没有】
12: 在哪里可以填地址,已经领取了,卡包也有半年权益卡了
13: 你得解决
14: 就是你们的白菜肯定不会喷甲醛吧
15: Produk emulsifikasi tubuh saya sudah beberapa hari ini tapi belum dikirimkan
16: 辛苦麻烦快点
17: Bantu kecepatan saja.
18: 好的,麻烦让他们赶快送来吧,谢谢
19: 好的,尽快,谢谢了
20: 放什么心,帮别人买的,都被别人催单好久了,烦
21: 我不想等太久了我下午还有事要做
22: 我昨天买的今天不是应该确认吗
23: 比抢火车票还厉害
24: 为什么之前可以借款
25: 什么鬼
26: 我够买的物料,显示五号就停止接单了,这又收到货了,怎么解释
27: 这么屌
28: 什么速度我靠
29: 哈哈哈哈??
30: 很高兴认识
31: 很高兴与您的对话哦
32: 女子我喜欢你。
33: 很高兴与您的对话哦
34: 膜拜膜拜,这个设计太棒了
35: 我觉得你挺帅
36: 她说:“今日收到两箱曲奇饼干,直接就是加一层薄薄的保鲜膜就发货了,盒子都烂了,这样的包装也是醉了”
37: 你们一直没给我回复呢
38: 平台和京东怎么差这么多钱啊
39: 这样太麻烦了,我需要到付,是厂家原因,为什么让客户麻烦
40: 怎么这么麻烦的呢
41: 老是不回复
42: 今天你的问题就没有给我解决好
43: 有投诉电话么,
44: 没还我倒立吃屎
45: 盗用我图片,在那恶心我,这不算吗?
46: 你们非要逼着我去投诉啊
47: 这个票,被废,非常恶劣。我相信你们供应也不可能去随便废,希望能有个合理的解释。
48: 不要官腔了可以吧,给我一个答复,什么时候才可以到,你们这是诈骗
49: 你这样我被扣了多少冤枉钱
50: 对的,非常感谢



1: 好嘅，好嘅，多谢大佬
2: 已经得咗，真系好感谢
3: 噢噢好啱，多谢你啦
4: 好嘅，多谢
5: 好嘅，多谢，下次再倾
6: 好嘅，麻烦你哋啦
7: 多谢你！！！
8: 都唔敢接
9: 好嘅，知晒啦，我嘅客人都以为货已经上架咗，就没事咗啦[表情029]
10: 查查呢个认证有冇
11: 但系我呢边真系冇
12: 哪度可以填地址，已经领咗，咩包都有半年嘅权益卡咗
13: 你要解决
14: 就系你哋嘅平价货肯定唔会喷甲醛吧
15: 我嘅身体乳已经几日都冇发货咗
16: 辛苦嘅，麻烦你快啲啦
17: 只系帮手快啲啫
18: 好嘅，麻烦你哋快啲送嚟，多谢
19: 好嘅，尽快啫，多谢
20: 咁啲心仲放得落去，系帮人买嘅，都畀人催单好耐咗，烦
21: 我唔想等好耐嘅，我下午仲有野要做
22: 我昨天买嘅，今日唔应该要确认咩吗
23: 比抢火车票仲犀利
24: 点解之前可以贷款
25: 乜嘢鬼
26: 我够买嘅物料，显示五号就停喺接单咗，但又收到货咗，点解解释
27: 咁犀利
28: 乜嘢速度，我靠
29: 哈哈哈哈??
30: 好开心识到你
31: 好开心同你倾
32: 女子我喜欢你。
33: 好开心同你倾
34: 膜拜膜拜，呢个设计真系好靓
35: 我觉得你好靓
36: 她话：“今日收到两箱曲奇饼，直接就系加一层薄薄嘅保鲜膜就发货咗，盒子都烂咗，咁样嘅包装真系醉咗”
37: 你哋一直都冇回复我呢
38: 平台同京东点解差咁多钱啊
39: 咁样真系好麻烦，我需要到付，系厂家原因，点解要客户麻烦
40: 点解咁麻烦嘅啊
41: 老係唔回复
42: 今日你嘅问题就冇解决好
43: 有投诉电话吗？
44: 没还我，我倒立食屎
45: 盗用我图片，喺度恶心我，咁样唔算吗？
46: 你哋非要逼我去投诉啊
47: 咁个票，被废咗，好恶劣。我相信你哋供应也不可能随便废，希望有個合理解释。
48: 不要官腔啦，畀我一个答复，点解先至到，你哋咁样系诈骗
49: 你咁样，我畀咗好多冤枉钱
50: 对嘅，非常感谢
Example({'label': '感激', 'text': '好嘅，好嘅，多谢大佬', 'source': '好哒,好哒,谢谢大大'}) (input_keys={'text'})
Example({'label': '感激', 'text': '已经得咗，真系好感谢', '

## 日语


In [44]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="日语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 日语: 
1: 谢谢您,新年快乐
2: 没啦,谢谢你哈
3: 好的,感谢,辛苦了
4: 哦,好吧,我试试看,谢了
5: 好的,我知道了,谢谢哈,那我和客户沟通一下
6: 好吧,等会我在问下别人,在来问你们把,现在没别的事情了,谢谢
7: 嗯嗯好的谢谢你了
8: 你发我
9: 麻烦请帮我取消订单,谢谢
10: 故障视频暂时没有
11: 能帮忙安排下重新退
12: 准备交年费,因为拍照识别手机号,不行,不敢交
13: 你给我发个苹果手机连接
14: 就感觉太干也可以换吗
15: 这个询价的麻烦尽快处理下
16: 能尽快处理吗,这样还让我们怎么玩
17: 我订的红酒只到了一份,另外一种怎么还没发货
18: 越快越好哈,嘿嘿
19: โปรดหายั่งเรียกด่วนๆครับ
20: Dapat mencari begitu lama?
21: Jadi agen tidak dapat melihat pesanan, sangat khawatir.
22: 延迟一天也不行?
23: 真牛了
24: 我明明点的是暂时关闭输入交易密码啊
25: 只是协商嘛,不是退吗?你刚才不是说退吗?
26: 不是吧?有没有搞错
27: 我够买的物料,显示五号就停止接单了,这又收到货了,怎么解释
28: 然后一直扣费?
29: 哈哈哈你好巧巧
30: 太好了ω
31: 爱你哦~
32: 要向你学习全心全意为人民服务的精神。又快又好
33: [emoji047]真好
34: 要向你学习全心全意为人民服务的精神。又快又好
35: Saya sangat menantikan.
36: 你好!为什么我报的明明是广州的全包班,合同里面是写明包了初考费和补考费的,但还要我自己缴费
37: 你说的不明不白
38: 我这个下午跟你们沟通得我也心累
39: 做了六单,最后这一单,验了几遍,没想到死在这一单上,很认真的在做
40: 可是一直打不通?/::<
41: 这也太慢了吧,有延迟险吗
42: 想着都烦
43: 算了，取关了，不会在消费你们家东西，辣鸡产品。
44: 你们找人威胁恐吓
45: 我草尼吗
46: 晚上十一二点,他们几个男的,商户自行给我更换了商品,我敢说啥
47: 他妈的你自己是傻子别把别人当傻子
48: 为什么提现还有这么多的条框,这个平台就是骗人的平台。
49

## 韩语


In [45]:
test_set,_ = stratified_sample(all_dataset, "label", 50,1)
import importlib
import transfer_lang

importlib.reload(transfer_lang)

trans = transfer_lang.TransferLang.from_dataset(test_set,lang="韩语")
ret = trans.do()
print(ret.content)
trans.update_set()

for item in test_set:
    print(item)
import importlib
import analysis

importlib.reload(analysis)

ana = analysis.Evaluate(data_set=test_set)
ret = ana.do()
print(ret)
ana.update_dataset()
ana.show_main_error()

把下面内容翻译成 韩语: 
1: 谢谢感恩!
2: 货找到谢谢
3: 多谢好的
4: 谢谢[emoji050]^_^
5: 谢谢您,不打扰你了
6: 没事了,我刚刚看到发货了谢谢
7: 好?,没事了,感谢
8: Pengaturan printout saya tiba-tiba tidak dapat dibuka
9: 我看吧还不正常的话给你们联系
10: 在那里抽呢
11: 点错支付宝了。付不了款
12: 显示已注册
13: 推广,然后有自己的二维码和联系方式
14: 那麻烦你帮我改一下吧,谢谢了
15: 三个小时了还没到
16: 好,赶快答复
17: 你好,很着急
18: 三天了都没到账
19: 让人好着急
20: 好,尽快。年前要解决
21: 稍微快一点,谢谢
22: 玩吧为什么最近总是闪退?
23: 流程还没走完?
24: 我明明点的是暂时关闭输入交易密码啊
25: 卧槽...他如果不还呢?
26: [emoji039]这么厉害的啊
27: 今天一看吓死还在计费
28: 今天一看吓死还在计费
29: 挺好的哈哈哈
30: 哈哈哈有意思好玩好开心呀!
31: 再次祝️您新年快乐
32: 嗯嗯。,认识你很高兴
33: [emoji047] Bagus sekali
34: 祝你小年愉快!
35: [emoji033]哈哈
36: 而且你们这个余额快用完时怎么也没短信通知呢?
37: 这句话我感觉很奇葩
38: 更换了手机和银行卡,为什么重新实名认证不了?
39: 有冲会员和美女聊天没有一个理
40: 这么低级的错误也犯
41: 每次跨服的人都不一样,这样算分是不是对我们本服的不公平
42: 确实你们这个收费有点理解不了
43: 我他妈刚借一小时啊
44: 我就不信了,你们想怎么就怎么,我会一直去投诉,我就不信了,没人管。
45: 屎奴屎鬼东西
46: 非常非常非常不好的体验
47: 投诉电话发来
48: 质量太差了的东东建议不要再卖了
49: 咨询你妹
50: 亲爱的,上午好!

1: 감사합니다!
2: 제품을 찾아서 감사합니다.
3: 감사합니다, 잘했어요.
4: 감사합니다 [emoji050]^_^
5: 감사합니다, 괜찮아요, 괜찮게 해주셔서.
6: 괜찮아요,我刚看到发货了，감사합니다.
7: 괜찮아요, 감사합니다.
8: 제 프린트아